**Study Case**

You are provided with a dataset comprising 3.4 million Google Store reviews for a music
streaming application (e.g., Spotify) sourced from various users. The management is currently
facing difficulties in extracting actionable insights from these reviews due to their unstructured
nature and the sheer volume of data. Manual handling has proven to be time-consuming, and a
more efficient solution is required to address management queries in a timely manner.


**Tasks**

As an AI engineer at our company, your task is to develop an internal Q&A tool capable of
extracting meaningful information from the Google Store reviews dataset (here) for the music
streaming application (Spotify, source). The tool should provide insightful responses to a variety
of management questions, such as:



```
Q: "What are the specific features or aspects that users appreciate the most in
our application?"
Example Response: "Users frequently praise the intuitive UI design, extensive
variety of music choices, and seamless user experience."
---
Q:"In comparison to our application, which music streaming platform are users
most likely to compare ours with?"
Example Response: "Users often draw comparisons with Pandora when discussing
our application's features and functionalities."
---
Q: "What are the primary reasons users express dissatisfaction with Spotify?"
Example Response: "Common concerns among dissatisfied users include occasional
playback issues, difficulties in discovering new music, and a desire for a more
personalized recommendation system."
---
Q: "Can you identify emerging trends or patterns in recent user reviews that
may impact our product strategy?"
Example Response: "Recent reviews highlight an increasing demand for enhanced
social sharing features, indicating a potential opportunity for improving the
platform's community engagement."

```



# Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q --upgrade google-generativeai langchain-google-genai python-dotenv
!pip install langchain
!pip install faiss
!pip install chromadb
!pip install langchain_community
!pip install langchain-huggingface
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing me

# Gemini API

In [3]:
from google.colab import userdata
import google.generativeai as genai

In [4]:
API_KEY = userdata.get('API_KEY')

genai.configure(api_key = API_KEY)

In [44]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-pro",
                               google_api_key=API_KEY,
                               temperature=0.9,
                               max_output_token=256
                               )

# Load Raw Data

In [6]:
import pandas as pd
import os

In [7]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Mekari Challenge/SPOTIFY_REVIEWS.csv'
df = pd.read_csv(dataset_path, delimiter =',')
df

,Unnamed: 0,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,0,14a011a8-7544-47b4-8480-c502af0ac26f,152618553977019693742,A Google user,Use it every day,5,1,1.1.0.91,2014-05-27 14:21:48
1,1,bfa8876b-470e-4640-83a7-77427f7f37e8,234382942865437071667,A Google user,"I enjoy the awesome UI of this app, and it has...",5,4,1.1.0.91,2014-05-27 14:36:02
2,2,70e8252f-058a-47d9-b066-df9e1571c970,174473604608358796368,A Google user,Love it! Especially the new design!,5,2,1.1.0.91,2014-05-27 14:40:01
3,3,672a155a-e81e-4d28-bdeb-a74c031bc072,286593453219054880269,A Google user,"Awesome UI, best music app out there!",5,1,1.1.0.91,2014-05-27 15:17:20
4,4,bbc1bf95-ed36-41a1-8b98-0f2e314caea5,167276875678680630145,A Google user,As a professional Android developer I'm glad t...,5,10,1.1.0.91,2014-05-27 15:26:48
...,...,...,...,...,...,...,...,...,...
3377418,3377418,41cafb32-c357-48df-97a3-e3867584f0d6,326345911143311484421,su************to,"Does not work, always says no internet but U t...",1,0,8.8.82.634,2023-11-15 23:11:02
3377419,3377419,ad68b04a-ff48-42f6-992b-bd765cdf9e94,322218874321066319125,Pe*******os,Goodjob,5,0,8.8.82.634,2023-11-15 23:11:29
3377420,3377420,9c36196d-12bc-4049-b7e4-27774a644bed,212802607434520579896,An*********ey,The blues always pull me through.,5,0,8.8.82.634,2023-11-15 23:11:32
3377421,3377421,ce45e1f8-78ad-40cc-a33b-d4e4672b8c78,415238355983315158902,St***en,awesome but you can't choose your song sometimes,4,0,8.8.82.634,2023-11-15 23:15:45


# Feature Engineering/ Data Cleansing

In [8]:
df = df.filter(['review_text', 'review_rating', 'review_likes', 'review_timestamp'])
df

,review_text,review_rating,review_likes,review_timestamp
0,Use it every day,5,1,2014-05-27 14:21:48
1,"I enjoy the awesome UI of this app, and it has...",5,4,2014-05-27 14:36:02
2,Love it! Especially the new design!,5,2,2014-05-27 14:40:01
3,"Awesome UI, best music app out there!",5,1,2014-05-27 15:17:20
4,As a professional Android developer I'm glad t...,5,10,2014-05-27 15:26:48
...,...,...,...,...
3377418,"Does not work, always says no internet but U t...",1,0,2023-11-15 23:11:02
3377419,Goodjob,5,0,2023-11-15 23:11:29
3377420,The blues always pull me through.,5,0,2023-11-15 23:11:32
3377421,awesome but you can't choose your song sometimes,4,0,2023-11-15 23:15:45


In [9]:
new_reviews = df[(df['review_timestamp'] >= '2020-01-01') & (df['review_timestamp'] <= '2023-12-31')]

In [10]:
new_reviews

,review_text,review_rating,review_likes,review_timestamp
1715746,Beautifull,5,0,2020-01-01 00:02:49
1715747,Ability to download music and listen in offlin...,5,0,2020-01-01 00:03:26
1715748,The best,5,0,2020-01-01 00:04:49
1715749,The best app to listen music.,5,0,2020-01-01 00:07:19
1715750,Luv spotify have the family plan,5,0,2020-01-01 00:11:40
...,...,...,...,...
3377418,"Does not work, always says no internet but U t...",1,0,2023-11-15 23:11:02
3377419,Goodjob,5,0,2023-11-15 23:11:29
3377420,The blues always pull me through.,5,0,2023-11-15 23:11:32
3377421,awesome but you can't choose your song sometimes,4,0,2023-11-15 23:15:45


In [11]:
insightful_reviews = new_reviews[new_reviews['review_likes'] >= 50]
insightful_reviews

,review_text,review_rating,review_likes,review_timestamp
1716495,"I edited my stars from 5 to 3 stars. Lately, t...",3,59,2020-01-01 19:13:28
1716940,Very good and does what it promises for the mo...,4,112,2020-01-02 06:42:41
1717315,I have been using this app since 2012 and love...,5,347,2020-01-02 15:25:41
1717349,"Always been a very big fan of Spotify, and it'...",4,177,2020-01-02 15:53:02
1717562,It's a really great app to listen to music and...,5,108,2020-01-02 21:03:27
...,...,...,...,...
3375164,This WAS my favorite music app for the last ye...,1,730,2023-11-14 17:14:29
3375641,App is actually unusable today. Nothing loads....,1,81,2023-11-15 01:06:05
3376294,"We pay for premium duo, but only one of the ac...",1,56,2023-11-15 09:09:29
3376643,It is a really good app. No where better to li...,4,51,2023-11-15 13:18:39




```
# 15695 Data @ >= 20 Likes
7913 Data @ >= 50 Likes
```
We will use the relatively insightful 7913 data as the base knowledge


In [12]:
# insightful_reviews.to_csv('insightful_reviews.csv', index=False)

## Checking Datas

In [13]:
appreciation_reviews = new_reviews[new_reviews['review_rating'] >= 4]
appreciation_reviews

,review_text,review_rating,review_likes,review_timestamp
1715746,Beautifull,5,0,2020-01-01 00:02:49
1715747,Ability to download music and listen in offlin...,5,0,2020-01-01 00:03:26
1715748,The best,5,0,2020-01-01 00:04:49
1715749,The best app to listen music.,5,0,2020-01-01 00:07:19
1715750,Luv spotify have the family plan,5,0,2020-01-01 00:11:40
...,...,...,...,...
3377413,I like the songs and I can listen to them wher...,4,0,2023-11-15 23:04:34
3377416,I LOVE Spotify but I am upset currently becaus...,4,0,2023-11-15 23:06:11
3377419,Goodjob,5,0,2023-11-15 23:11:29
3377420,The blues always pull me through.,5,0,2023-11-15 23:11:32


In [14]:
improvement_reviews = new_reviews[new_reviews['review_rating'] <= 3]
improvement_reviews

,review_text,review_rating,review_likes,review_timestamp
1715757,"Was good for first week. Then, plays few songs...",1,0,2020-01-01 00:20:40
1715770,Keeps cutting out,1,0,2020-01-01 00:45:29
1715780,When i got this app it was fine until it would...,1,0,2020-01-01 01:05:22
1715798,"TO MANY ADDS. 8 minute is not 30 spodify, stop...",2,0,2020-01-01 01:23:15
1715801,Just download your music,1,0,2020-01-01 01:24:49
...,...,...,...,...
3377414,The new like / add to a playlist feature is no...,3,0,2023-11-15 23:05:35
3377415,bakwasss app,1,0,2023-11-15 23:06:03
3377417,"Bring back liked songs button, new add button ...",2,0,2023-11-15 23:06:56
3377418,"Does not work, always says no internet but U t...",1,0,2023-11-15 23:11:02


In [15]:
appreciation_reviews[(appreciation_reviews['review_rating'] >= 5) & (appreciation_reviews['review_likes'] >= 50)]

,review_text,review_rating,review_likes,review_timestamp
1717315,I have been using this app since 2012 and love...,5,347,2020-01-02 15:25:41
1717562,It's a really great app to listen to music and...,5,108,2020-01-02 21:03:27
1720969,Such a good music streaming service. Bought 6 ...,5,302,2020-01-06 06:04:47
1721679,I didn't think I would ever use a music stream...,5,1070,2020-01-07 00:10:13
1723849,Awesome...! I enjoy listening to Spotify bette...,5,343,2020-01-09 00:52:37
...,...,...,...,...
3358840,The pros of Spotify far outweigh the cons. Hav...,5,219,2023-11-07 04:25:58
3366148,It's great but only for premium users,5,614,2023-11-10 09:15:39
3367635,By far the best music streaming app. I don't k...,5,60,2023-11-10 23:10:39
3369302,Great app. I did not realize how great it was ...,5,55,2023-11-11 16:07:12


In [16]:
indices = appreciation_reviews[appreciation_reviews['review_likes'] == 100].index
for index in indices:
    print(appreciation_reviews.loc[index, 'review_text'])

I have been a Spotify user for years for a reason. Reasonably price and easy to use, I get access to music I love and the occasional rad recommendation that leads to an ever-growing library. When I meet new people who don't have Spotify, I convert them to a clearly superior music streaming platform. Honestly, at this point, Spotify owes me money!
Spotify is brilliant, never looked back after downloading this, not only good with music you love, but also once it gets to know you, with the music or mixed albums its creates for you. I still use the free version as it suits my needs at this point, I always suggest it to everyone as it really is very good... i promise you wont be disappointed...
I have been using Apple Music, Deezer and Yandex.Music before. And Spotify is the best music app for me! It works perfect, has the biggest library of the music and has the greatest design. The multi-device sync is crazy, cause I can control music playing on one device using another and I use often So

In [17]:
indices = improvement_reviews[improvement_reviews['review_timestamp'] == '2023-11-15 19:25:47'].index
for index in indices:
    print(improvement_reviews.loc[index, 'review_text'])

This most recent update, November 2023, is terrible. Removal of the heart is less than helpful. I don't add every song to my liked songs (I mostly skip instrumentals) and now I can't easily check if a song is added to Liked Songs or not. Very frustrating. Just add better playlist functionality, like checking which playlists a song is listed in, instead of this garbage. The heart was clear and useful. This is not.


In [18]:
ins_appreciation_reviews = appreciation_reviews[appreciation_reviews['review_likes'] >= 50]
ins_appreciation_reviews

,review_text,review_rating,review_likes,review_timestamp
1716940,Very good and does what it promises for the mo...,4,112,2020-01-02 06:42:41
1717315,I have been using this app since 2012 and love...,5,347,2020-01-02 15:25:41
1717349,"Always been a very big fan of Spotify, and it'...",4,177,2020-01-02 15:53:02
1717562,It's a really great app to listen to music and...,5,108,2020-01-02 21:03:27
1720969,Such a good music streaming service. Bought 6 ...,5,302,2020-01-06 06:04:47
...,...,...,...,...
3367747,Spotify is a very good app for listening to bo...,4,52,2023-11-11 01:47:57
3369302,Great app. I did not realize how great it was ...,5,55,2023-11-11 16:07:12
3373964,"Good for now 👍, wish it was more similar to th...",4,1822,2023-11-14 06:11:32
3374233,The new update has a weird change but alot mor...,5,171,2023-11-14 08:55:02


In [19]:
ins_improvement_reviews = improvement_reviews[improvement_reviews['review_likes'] >= 50]
ins_improvement_reviews

,review_text,review_rating,review_likes,review_timestamp
1716495,"I edited my stars from 5 to 3 stars. Lately, t...",3,59,2020-01-01 19:13:28
1718255,I've had Spotify for some time but until last ...,1,474,2020-01-03 14:00:13
1720223,I've been using Spotify premium for months now...,1,51,2020-01-05 12:36:46
1722279,"So I was using it fine, no problems, until I g...",1,55,2020-01-07 15:20:39
1723161,"The app is decent, but I think it's time you f...",2,110,2020-01-08 12:24:33
...,...,...,...,...
3375161,Would have rated 5 stars but the amount of ads...,3,157,2023-11-14 17:13:18
3375164,This WAS my favorite music app for the last ye...,1,730,2023-11-14 17:14:29
3375641,App is actually unusable today. Nothing loads....,1,81,2023-11-15 01:06:05
3376294,"We pay for premium duo, but only one of the ac...",1,56,2023-11-15 09:09:29


# Set Up Databases

In [12]:
from langchain.document_loaders import DataFrameLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

In [13]:
loader = DataFrameLoader(insightful_reviews, page_content_column="review_text")

In [14]:
documents = loader.load()
documents

[Document(metadata={'review_rating': 3, 'review_likes': 59, 'review_timestamp': '2020-01-01 19:13:28'}, page_content="I edited my stars from 5 to 3 stars. Lately, this app just stops playing, which is really annoying. Especially when I am in the car and I have google maps on, and after receiving directions spotify just stops playing music. I'm paying for this app which I probably will cancel if it is not fixed soon. - Really love this app! I was looking for German music, which I couldn't find on other apps. Here I found a treasure chest of it, could use more of it, but still love it!"),
 Document(metadata={'review_rating': 4, 'review_likes': 112, 'review_timestamp': '2020-01-02 06:42:41'}, page_content="Very good and does what it promises for the most part. The only reason why I can't give it a five star is because every once in a while, it will offer the ad for 30- minutes ad free music. Probably 20% of the times I watch the ad, I still have ads, some as soon as 5 minutes after suppos

In [15]:
len(documents)

7913

In [16]:
split_texts = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
reviews = split_texts.split_documents(documents)

In [17]:
len(reviews)

7913

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.docstore.document import Document
from langchain_huggingface import HuggingFaceEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [20]:
vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)

In [21]:
len(vectorstore.get()['documents'])

7913

# Set Up Retriever

In [22]:
from re import search
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 100, "fetch_k": 1000})

In [23]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7ba86221fc40>, search_type='mmr', search_kwargs={'k': 100, 'fetch_k': 1000})

# Prompt Engineering

In [45]:
prompt_template = """
  You are a friendly Q&A Chatbot capable of extracting meaningful information from the reviews data of the music streaming application: Spotify.
  You should provide clear, concise, and insightful responses to a variety of management questions.
  Since you're analyzing for company's executives, summarize your answers with insights and provide strong reasoning for your answers.
  Summarize your response, for example: "Our app is better because of the ..."
  Don't provide the wrong answer\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

# Test Area

In [29]:
question = "What are the specific features or aspects that users appreciate the most in our application?"
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
docs = retriever.invoke(question)
response = chain.invoke(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)
response

{'output_text': 'Users appreciate the extensive music library, personalized recommendations, seamless playback, and user-friendly interface. They also value the ability to create and share playlists, discover new music, and connect with other users.'}

In [46]:
question_2 = "In comparison to our application, which music streaming platform are users most likely to compare ours with?"
chain_2 = load_qa_chain(model, chain_type="stuff", prompt=prompt)
docs_2 = retriever.invoke(question_2)
response_2 = chain_2.invoke(
    {"input_documents":docs_2, "question": question_2}
    , return_only_outputs=True)
response_2

{'output_text': 'Users have found that Spotify is the most comparable app to our application.'}

In [30]:
question_3 = "What are the primary reasons users express dissatisfaction with Spotify?"
chain_3 = load_qa_chain(model, chain_type="stuff", prompt=prompt)
docs_3 = retriever.invoke(question_3)
response_3 = chain_3.invoke(
    {"input_documents":docs_3, "question": question_3}
    , return_only_outputs=True)
response_3

{'output_text': "Based on the reviews, users have expressed dissatisfaction with Spotify due to several reasons:\n\n* **Bugs and technical issues:** Many users complain about frequent bugs, glitches, and connectivity problems that disrupt their listening experience.\n\n* **User interface (UI) changes:** Spotify's frequent UI updates have been met with criticism, with users finding them confusing, cluttered, and less user-friendly than previous versions.\n\n* **Limited features:** Some users have expressed frustration with the removal of certain features, such as the ability to view lyrics, create custom radio stations, or control playback speed.\n\n* **Excessive advertising:** Free users of Spotify complain about the high frequency of advertisements, which they find intrusive and disruptive.\n\n* **Lack of artist compensation:** Some users have criticized Spotify's payment model, arguing that it does not fairly compensate artists for their work.\n\n* **Content moderation:** Spotify's d

In [42]:
question_4 = "Can you identify emerging trends or patterns in recent user reviews that may impact our product strategy?"
chain_4 = load_qa_chain(model, chain_type="stuff", prompt=prompt)
docs_4 = retriever.invoke(question_3)
response_4 = chain_4.invoke(
    {"input_documents":docs_4, "question": question_4}
    , return_only_outputs=True)
response_4

{'output_text': "Recent user reviews reveal several emerging trends and patterns that may impact Spotify's product strategy:\n\n1. **Growing dissatisfaction with the user experience**: Many users are expressing frustration with the app's frequent bugs, glitches, and poor performance. They also criticize the recent redesign of the user interface, which they find confusing and difficult to navigate.\n\n2. **Declining satisfaction with the music library**: Users are increasingly complaining about the limited selection of music available on Spotify, particularly in certain genres and regions. They also express concerns about the removal of certain songs and albums from the platform.\n\n3. **Increasing competition from rival streaming services**: Several users mention switching to other streaming services, such as Apple Music or YouTube Music, due to dissatisfaction with Spotify's app or music library. This suggests that Spotify faces growing competition in the music streaming market.\n\n4.

# Streamlit

In [ ]:
import streamlit as st




# 2nd Method Testing

In [33]:
# QA = RetrievalQA.from_chain_type(llm=model, chain_type='stuff', retriever=retriever, chain_type_kwargs={"prompt": prompt})

In [47]:
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | model
# )

In [48]:
# question = "What are the specific features or aspects that users appreciate the most in our application?"

In [49]:
# QA.invoke(question)

{'query': 'What are the specific features or aspects that users appreciate the most in our application?',
 'result': 'The users of the app appreciate the wide variety of music available, the ability to create their own playlists, and the ease of use. They also appreciate that the app is available on multiple devices and that it is constantly being updated with new features.'}

In [50]:
# question_2 = "In comparison to our application, which music streaming platform are users most likely to compare ours with?"

In [51]:
# QA.invoke(question_2)

{'query': 'In comparison to our application, which music streaming platform are users most likely to compare ours with?',
 'result': "Users often compare Spotify to YouTube Music. They appreciate the vast selection of music and the user-friendly interface of both apps. However, some users have expressed frustration with Spotify's lack of an equalizer on Android and desktop devices, as well as its occasional slow loading and searching issues. Additionally, some users have noticed a decline in sound quality and reliability when streaming to other devices. Despite these concerns, Spotify remains a popular choice due to its extensive content library, collaborative playlist tools, and robust discovery features."}

In [ ]:
# question_3 = "What are the primary reasons users express dissatisfaction with Spotify?"

In [ ]:
# QA.invoke(question_3)

In [ ]:
# question_4 = "What is the platform sentiment for 2022?"

In [ ]:
# QA.invoke(question_4)

In [ ]:
# insightful_reviews['text'] = insightful_reviews.apply(lambda row: f"User review: [{row['review_text']}]. Date: {row['review_timestamp']}. Rating: {row['review_rating']}. Likes: {row['review_likes']}.", axis=1)
# texts = df['text'].tolist()